In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from matplotlib import rcParams, pyplot as plt
import numpy as np
import pandas as pd
from pathlib import Path
import seaborn as sns
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import cross_val_score
from warnings import simplefilter
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from xgboost import plot_importance
%matplotlib inline

In [3]:
rcParams['figure.figsize'] = (16, 8)
plt.style.use('fivethirtyeight')
simplefilter('ignore')

In [4]:
data_dir = Path('input/')
sub_dir = Path('sub/')

trn_file = data_dir/'train.csv'
tst_file = data_dir/'test.csv'
sample_file = data_dir/'sample_submission.csv'

target_col = 'class'
seed=42

In [5]:
algo_name = 'dt'
feature_name = 'j1'
model_name = f'{algo_name}_{feature_name}'

sub_file = sub_dir / f'{model_name}.csv'

In [6]:
trn = pd.read_csv(trn_file, index_col =0)
tst = pd.read_csv(tst_file, index_col =0)
X = trn.iloc[:,:18]
y = trn[target_col]

print(y.shape, X.shape, tst.shape)
sub_file = sub_dir/f'{model_name}.csv'


(320000,) (320000, 18) (80000, 18)


In [13]:
trn[target_col]

id
0         0
1         1
2         0
3         0
4         0
         ..
319995    2
319996    1
319997    0
319998    0
319999    0
Name: class, Length: 320000, dtype: int64

In [7]:
X = np.array(X).reshape(320000,-1)
y = np.array(y).reshape(320000,1)

In [8]:
rskf = RepeatedStratifiedKFold(n_splits=3, n_repeats=3 , random_state=1)

In [9]:
for train_index, test_index in rskf.split(X,y):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

TRAIN: [     0      1      2 ... 319996 319998 319999] TEST: [     9     13     15 ... 319992 319993 319997]
TRAIN: [     1      3      4 ... 319996 319997 319999] TEST: [     0      2      5 ... 319987 319995 319998]
TRAIN: [     0      2      5 ... 319995 319997 319998] TEST: [     1      3      4 ... 319994 319996 319999]
TRAIN: [     0      1      2 ... 319997 319998 319999] TEST: [     6      7      8 ... 319991 319995 319996]
TRAIN: [     1      2      3 ... 319995 319996 319998] TEST: [     0      4      9 ... 319994 319997 319999]
TRAIN: [     0      4      6 ... 319996 319997 319999] TEST: [     1      2      3 ... 319990 319993 319998]
TRAIN: [     0      1      4 ... 319996 319998 319999] TEST: [     2      3     10 ... 319987 319988 319997]
TRAIN: [     2      3      5 ... 319990 319991 319997] TEST: [     0      1      4 ... 319996 319998 319999]
TRAIN: [     0      1      2 ... 319997 319998 319999] TEST: [     5      7      8 ... 319985 319990 319991]


In [1]:
# max_depth와 min_child_weight를 튜닝
param_test1 = {
    'max_depth':range(3,10,3),
     'min_child_weight':range(1,6,2)
}
gsearch1 = GridSearchCV(estimator = xgb.XGBClassifier(
    learning_rate =0.1,
    n_estimators=1000,
    max_depth=5,
    min_child_weight=1,
    gamma=0,
    subsample=0.8,
    colsample_bytree=0.8,
    objective= 'multi:softmax',
    nthread=-1,
    scale_pos_weight=1,
    seed=42
),
param_grid = param_test1, scoring='accuracy',n_jobs=-1,iid=False, cv=rskf, verbose=10)
gsearch1.fit(X_train,y_train)
gsearch1.cv_results_, gsearch1.best_params_, gsearch1.best_score_

NameError: name 'GridSearchCV' is not defined

In [18]:
#  Gamma를 튜닝
param_test2 = {
 'gamma':[i/10.0 for i in range(0,5)]
}
gsearch2 = GridSearchCV(estimator = XGBClassifier(learning_rate =0.1, 
                                                  n_estimators=1000, 
                                                  max_depth=3,
                                                  min_child_weight=5, 
                                                  gamma=0, 
                                                  subsample=0.8, 
                                                  colsample_bytree=0.8,
                                                  objective= 'multi:softmax', 
                                                  thread=-1, 
                                                  scale_pos_weight=1,
                                                  seed=2019), 
                        param_grid = param_test2, scoring='accuracy', n_jobs=-1, iid=False, cv=5)
gsearch2.fit(train[predictors],train[target])
gsearch2.cv_results_, gsearch2.best_params_, gsearch2.best_score_

Fitting 5 folds for each of 9 candidates, totalling 45 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed: 88.7min
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed: 209.3min
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed: 209.3min


KeyboardInterrupt: 

In [ ]:
# subsample and colsample_bytree를 튜닝
param_test3 = {
 'subsample':[i/10.0 for i in range(6,10)],
 'colsample_bytree':[i/10.0 for i in range(6,10)]
}
gsearch3 = GridSearchCV(estimator = XGBClassifier(learning_rate =0.1, 
                                                  n_estimators=1000, 
                                                  max_depth=3,
                                                  min_child_weight=5, 
                                                  gamma=, 
                                                  subsample=0.8, 
                                                  colsample_bytree=0.8,
                                                  objective= 'multi:softmax'
                                                  thread=-1, 
                                                  scale_pos_weight=1,
                                                  seed=2019), 
                        param_grid = param_test3, scoring='accuracy', n_jobs=-1, iid=False, cv=5, verbose=10)
gsearch3.fit(train[predictors],train[target])
gsearch3.cv_results_, gsearch3.best_params_, gsearch3.best_score_

In [ ]:
#subsample 추가 튜닝하기
param_test4 = {
 'subsample':[i/100.0 for i in range(40,80)],
}
gsearch4 = GridSearchCV(estimator = XGBClassifier(learning_rate =0.1, 
                                                  n_estimators=1000, 
                                                  max_depth=3,
                                                  min_child_weight=5, 
                                                  gamma=0, 
                                                  subsample=, 
                                                  colsample_bytree=0.8,
                                                  objective= 'multi:softmax', 
                                                  thread=-1, 
                                                  scale_pos_weight=1,
                                                  seed=2019), 
                        param_grid = param_test4, scoring='accuracy', n_jobs=-1, iid=False, cv=5, verbose=10)
gsearch4.fit(train[predictors],train[target])
gsearch4.cv_results_, gsearch4.best_params_, gsearch4.best_score_

In [ ]:
# Regularization Parameter 튜닝
param_test5 = {
 'reg_alpha':[1e-5, 1e-2, 0.1, 1, 100]
}
gsearch5 = GridSearchCV(estimator = XGBClassifier(learning_rate =0.1, 
                                                  n_estimators=1000, 
                                                  max_depth=3,
                                                  min_child_weight=5, 
                                                  gamma=0, 
                                                  subsample=, 
                                                  colsample_bytree=0.8,
                                                  objective= 'multi:softmax', 
                                                  thread=-1, 
                                                  scale_pos_weight=1,
                                                  seed=2019), 
                        param_grid = param_test5, scoring='accuracy', n_jobs=-1, iid=False, cv=5, verbose=10)
gsearch5.fit(train[predictors],train[target])
gsearch5.cv_results_, gsearch5.best_params_, gsearch5.best_score_

In [16]:
# learning rate 감소
predictors = [x for x in X_train.columns if x not in [target]]
xgb1 = XGBClassifier(
    learning_rate =0.01,
    n_estimators=5000,
    max_depth=3,
    min_child_weight=5,
    gamma=0,
    reg_alpha=,
    subsample=0.67,
    colsample_bytree=0.8,
    objective= 'multi:softmax',
    nthread=-1,
    scale_pos_weight=1,
    seed=2019
)
modelfit(xgb1, train, predictors)

SyntaxError: invalid syntax (<ipython-input-16-7b2cfe75c3e4>, line 9)

In [ ]:
scores = cross_val_score(model, X, y, scoring='accuracy', cv=rskf, n_jobs=-1)

In [ ]:
print("Accuracy : %.3f(%.3f)"%(np.mean(scores),np.std(scores)))

In [ ]:
sub = pd.read_csv(sample_file, index_col = 0)
sub.head()

In [ ]:
pred = model.predict(tst)
submission['class'] = pred
submission.to_csv("submission.csv",index=False, encoding='utf-8-sig')